In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
import networkx as nx

In [ ]:
from ema_workbench import (Model, MultiprocessingEvaluator, RealParameter,  CategoricalParameter, IntegerParameter, RealParameter, ScalarOutcome, TimeSeriesOutcome, Policy, Scenario, ema_logging, Constant)

from ema_workbench.em_framework.evaluators import perform_experiments, Samplers, LHSSampler, SobolSampler,MorrisSampler
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.analysis import feature_scoring, parcoords
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from problem_formulation import get_model_for_problem_formulation

import time

from SALib.analyze import sobol

In [ ]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

In [ ]:
from dike_model_function import DikeNetwork  # @UnresolvedImport

def sum_over(*args):
    return sum(args)

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [ ]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [ ]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [ ]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

In [16]:
#running the model through EMA workbench
ema_logging.log_to_stderr(ema_logging.INFO)

#open exploration with 4 policies, random
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)


[MainProcess/INFO] pool started with 16 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:42<00:00,  4.71it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [17]:
#observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,92,291.056735,10,0.989256,327.166666,1.0,0.235870,160.966070,1.5,0.833672,...,10,2,6,8,1,7,0,4,0,dikesnet
1,109,275.440636,10,0.471762,199.945526,1.0,0.888496,95.439077,10,0.220239,...,10,2,6,8,1,7,0,5,0,dikesnet
2,66,81.818186,1.0,0.386689,178.301024,10,0.772782,212.303368,1.0,0.054542,...,10,2,6,8,1,7,0,6,0,dikesnet
3,59,262.688779,10,0.247307,166.196041,1.0,0.686789,42.823433,1.0,0.759803,...,10,2,6,8,1,7,0,7,0,dikesnet
4,55,74.139748,1.5,0.282172,163.617240,10,0.462925,80.991712,1.5,0.188729,...,10,2,6,8,1,7,0,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,48,260.357148,1.0,0.210712,92.311258,1.0,0.093028,243.192281,1.0,0.208908,...,1,1,2,0,8,1,4,49,3,dikesnet
196,96,192.217283,1.0,0.514107,33.602667,10,0.569630,185.669064,10,0.257574,...,1,1,2,0,8,1,4,50,3,dikesnet
197,122,174.426556,1.5,0.374380,269.508840,1.5,0.932860,154.080586,10,0.852692,...,1,1,2,0,8,1,4,51,3,dikesnet
198,105,148.382662,1.5,0.808039,134.062635,1.0,0.011585,207.003346,1.5,0.097246,...,1,1,2,0,8,1,4,52,3,dikesnet


In [ ]:
outcomes['A.1 Total Costs'].shape
problem = get_SALib_problem(uncertainties)
print(problem)

In [ ]:
#SOBOL SENSITIVITY ANALYSIS
#to see how much the uncertainties add to the variance of the outcomes

with MultiprocessingEvaluator(dike_model) as evaluator:
   sobol_results = evaluator.perform_experiments(scenarios=50, policies=4, uncertainty_sampling=Samplers.SOBOL)
experiments_sobol, outcomes_sobol = sobol_results

In [ ]:
#making a list of the outcome names
outcome_options = ['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths','RfR Total Costs', 'Expected Evacuation Costs']

problem = get_SALib_problem(dike_model.uncertainties)

#making a sobol table and sobol figure that shows how much the variable contributes to the variance on its own (first order effects = S1) and contributes when including interactions with other inputs (total effects = ST)
for option in outcome_options:
    outcome_sobol = np.squeeze(outcomes_sobol[option])
    Si = sobol.analyze(problem, outcome_sobol, calc_second_order=True, print_to_console=False)
    Si_filter = {k: Si[k] for k in ["ST", "ST_conf", "S1", "S1_conf"]}
    Si_df = pd.DataFrame(Si_filter, index=problem["names"])
    print(option)
    print(Si_df)
    sns.set_style('white')
    fig, ax = plt.subplots(1)

    indices = Si_df[['S1','ST']]
    err = Si_df[['S1_conf','ST_conf']]

    indices.plot.bar(yerr=err.values.T,ax=ax)
    fig.set_size_inches(8,6)
    fig.subplots_adjust(bottom=0.3)
    plt.show()


In [ ]:
#MORDM - Many objectives robust decision making
#All the outcomes need to be minimized in the ideal situation, because all types of costs and all deaths should be as low as possible
dike_model.outcomes = [ScalarOutcome('A.1 Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.1_Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.2 Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.2_Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.3 Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.3_Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.4 Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.4_Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.5 Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.5_Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('RfR Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('Expected Evacuation Costs', kind=ScalarOutcome.MINIMIZE),
                       ]

In [ ]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(epsilons=[0.1, 0.1], nfe=100)

In [ ]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{'0_RfR 0':1,
                                  '0_RfR 1':1,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':5}),
           Policy('policy 2', **{'4_RfR 0':1,
                                  '4_RfR 1':1,
                                  '4_RfR 2':1,
                                  'A.5_DikeIncrease 0':5}),
           Policy('policy 3', **{'1_RfR 0':1,
                                  '2_RfR 1':1,
                                  '3_RfR 2':1,
                                  'A.3_DikeIncrease 0':5})]

In [ ]:
#pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started with 16 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
  0%|                                                  | 0/300 [00:00<?, ?it/s]